## Building a Chat Bot

In the examples below, we will build a very simple application that creates a simple chat bot. The bot is monitoring the actions that are happening on Slack, and posts some acknowledgement messages every time someone types a message.


#### Authentication

We start first by authenticating our application. (See notes in S1). We read the `slack_secret.json` file, and we instantiate then our SlackClient.

In [ ]:
# Read the access token from the file
import json

secrets_file = 'slack_secret.json'
f = open(secrets_file, 'r') 
content = f.read()
auth_info = json.loads(content)
token = auth_info["access_token"]
f.close()

In [ ]:
# If you happen not to have the slackclient library installed, 
# you can uncomment and run the code below
# !sudo python3 -m pip install -U slackclient

In [ ]:
from slackclient import SlackClient
sc = SlackClient(token)

In [ ]:
import time

if sc.rtm_connect():
    # We are going to be polling the Slack API for recent events continuously
    while True:
        # We are going to wait 1 second between monitoring attempts
        time.sleep(1)
        # If there are any new events, we will get a response. If there are no events, the response will be empty
        response = sc.rtm_read()
        for item in response:
            # We print the event on the screen for informational purposes
            print(item) 
            event_type = item.get("type")
            if event_type == 'message':
                print(item)
                message = "Thank you user {u} for participating in channel {c}".format(u=item["user"], c=item["channel"])
                sc.api_call("chat.postMessage", channel="#bots", text=message)